In [7]:
import cv2
import time
import os
import threading
from threading import Thread
from time import sleep
import numpy as np
import detector1
import detector2
import detector3
import detector4
import detector5
from detector3 import ImgNet

ModuleNotFoundError: No module named 'yolov5'

In [ ]:
# !pip install -r C:/Users/maria/Desktop/hackaton_final/hackaton/requirements.txt

In [6]:
d1 = detector1.Predictor() # предиктор рамок вокруг лица, на чистом CV
# d2 = detector2.Predictor() # предиктор отслеживания изменения фона на фото и окружающе сцены, модель RandomForest
d3 = detector3.Predictor() # предиктор с механизмом внимания
d4 = detector4.Predictor() # предиктор телефона(планшета, ноута) в кадре, на облегченной Yolo
d5 = detector5.Predictor() # предиктор цветового балланса в кадре, на DecisionTree

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

In [3]:
# служебные функции
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    if image is None: return None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized
def avg(arr):
    arr=[x for x in arr if not x is None]
    return np.average(arr)

In [20]:
# главная функция. ее нужно запускать для теста. Аргумент путь до односекундного видео. 
# Возвращает вероятность атаки предъявлением (0 - точно нет атаки, 1 - атака точно есть) и время обработки видео
def main(one_sec_video):
    cap = cv2.VideoCapture(one_sec_video)
    rez=np.array([None,None,None,None,None,None,])
    threads=[]
    images=[]
    images_=[]
    cnt=0
    while True:
        ret, img_ = cap.read()
        cnt+=1
        if cnt % 8!=0: continue
        img=image_resize(img_,width=450)
        if not ret or len(images)>=3: break
        images.append(img)
        images_.append(img_)
    start=time.time() 
    threads.append(Thread(target=d1.predict, args=(images, rez, 1,))) #  можно закомментирвать две строки, чтобы отключить предиктор
    threads[-1].start()  
    threads.append(Thread(target=d2.predict, args=(images, rez, 2,)))  #  можно закомментирвать две строки, чтобы отключить предиктор
    threads[-1].start() 
    threads.append(Thread(target=d3.predict, args=(images_, rez, 3,)))  #  можно закомментирвать две строки, чтобы отключить предиктор
    threads[-1].start() 
    threads.append(Thread(target=d4.predict, args=(images_, rez, 4,)))  #  можно закомментирвать две строки, чтобы отключить предиктор
    threads[-1].start() 
    threads.append(Thread(target=d5.predict, args=(images_, rez, 5,)))  #  можно закомментирвать две строки, чтобы отключить предиктор
    threads[-1].start() 
    for i in range(100):
       cnt=0 
       for t in threads:
           cnt+= 1 if not t is None and t.is_alive() else 0
       if cnt==0: break    
       sleep(0.01)
    rez_= avg(rez)
    stop=time.time() 
    print(f"Probability attack: {rez_}   Time predict: {stop-start}s")
    return rez_,stop-start

# Далее тестирование на золотом стандарте

In [21]:
attack_files = [f for f in os.listdir("video4test/attack/") if os.path.getsize(os.path.join("video4test/attack/", f)) > 0 and not f.startswith('.')]
noattack_files = [f for f in os.listdir("video4test/noattack/") if os.path.getsize(os.path.join("video4test/noattack/", f)) > 0 and not f.startswith('.')]
target=np.array([])
predict=np.array([])
filenames=np.array([])
for f in attack_files: 
    # print("video4test/attack/"+f)
    p,_=main("video4test/attack/"+f)
    predict=np.append(predict,p)
    target=np.append(target,1)
    filenames = np.append(filenames,"video4test/attack/"+f)
for f in noattack_files: 
    # print("video4test/noattack/"+f)
    p,_=main("video4test/noattack/"+f)    
    predict=np.append(predict,p)
    target=np.append(target,0)
    filenames = np.append(filenames,"video4test/noattack/"+f)

Probability attack: 0.7969543099403381   Time predict: 0.3078446388244629s
Probability attack: 0.3341052969296773   Time predict: 0.9312222003936768s
Probability attack: 0.9313953757286072   Time predict: 0.787980318069458s
Probability attack: 0.8626976569493612   Time predict: 0.9897019863128662s
Probability attack: 0.5992941975593566   Time predict: 0.833214282989502s
Probability attack: 0.5130835612614949   Time predict: 0.9253144264221191s
Probability attack: 0.2533447156349818   Time predict: 0.8562972545623779s
Probability attack: 0.6864746252695719   Time predict: 1.0887954235076904s
Probability attack: 0.896593976020813   Time predict: 0.6206109523773193s
Probability attack: 0.058798531691233315   Time predict: 0.8298413753509521s
Probability attack: 0.36573126713434856   Time predict: 0.8393843173980713s
Probability attack: 0.854941725730896   Time predict: 0.9256396293640137s
Probability attack: 0.7959987322489421   Time predict: 0.92240309715271s
Probability attack: 0.845156

Exception in thread Thread-1069 (predict):
Traceback (most recent call last):
  File "/usr/lib64/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/test/hackaton/detector2.py", line 78, in predict
    arr_rez[idx]=  answersSum/count    
ZeroDivisionError: float division by zero


Probability attack: 0.26105796794096625   Time predict: 0.4887082576751709s
Probability attack: 0.5315995931625366   Time predict: 0.9776890277862549s
Probability attack: 0.7281978368759155   Time predict: 0.8476011753082275s
Probability attack: 0.7248417854309082   Time predict: 0.8320250511169434s
Probability attack: 0.6661468664805094   Time predict: 0.8621344566345215s
Probability attack: 0.7978778719902039   Time predict: 0.598642110824585s
Probability attack: 0.3946702480316162   Time predict: 0.8322477340698242s
Probability attack: 0.4495607654253642   Time predict: 0.9533698558807373s
Probability attack: 0.5959360321362813   Time predict: 0.8908495903015137s
Probability attack: 0.3977714935938518   Time predict: 0.7730984687805176s
Probability attack: 0.7098598480224609   Time predict: 0.7878401279449463s
Probability attack: 0.5637403567632039   Time predict: 0.7776002883911133s
Probability attack: 0.6430402636528015   Time predict: 0.8771622180938721s
Probability attack: 0.393

In [25]:
THRESHOLD=0.4
correct_predicts=sum((predict > THRESHOLD).astype(int) == target)
print(f"Правильно предсказано: {correct_predicts} из {len(target)}")
true_predict=(predict > THRESHOLD).astype(int) == target
print(f"Ошибка в обнаружении в видео: {filenames[np.logical_not(true_predict)]}")

Правильно предсказано: 75 из 92
Ошибка в обнаружении в видео: ['video4test/attack/IMG_0412.MOV' 'video4test/attack/20231120_091505.mp4' 'video4test/attack/20231120_091444.mp4' 'video4test/attack/20231120_091932.mp4' 'video4test/attack/20231120_092154.mp4' 'video4test/attack/pr8.mp4' 'video4test/attack/r5.mp4' 'video4test/attack/pr4.mp4' 'video4test/attack/pr5.mp4'
 'video4test/attack/20231114_161627_05.mp4' 'video4test/attack/20231120_091426.mp4' 'video4test/attack/20231120_091834.mp4' 'video4test/attack/20231120_091815.mp4' 'video4test/noattack/noR5.mp4' 'video4test/noattack/3331_.mp4' 'video4test/noattack/noR7.mp4' 'video4test/noattack/video_4_1.mp4']


In [26]:
print(f"Acc: {correct_predicts/len(target)}")

Acc: 0.8152173913043478


КОНЕЦ

In [24]:
main("video4test/attack/20231114_161627_04.mp4")

Probability attack: 0.7564778566360474   Time predict: 0.9863269329071045s


(0.7564778566360474, 0.9863269329071045)

In [ ]:
# p=detector5.Predictor()
# img=cv2.imread("1.jpg")
# rez=p.predict([img],np.zeros(6),1)